# RAG Based on Hybrid Search (Lexical + Semantic)


## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [5]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 1024
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f351bdaed70>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 1024}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f3559c7f2e0>])

### Embedding 모델 선택

In [7]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [8]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [9]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. LangChainOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [10]:
from utils.proc_docs import get_parameter

In [11]:
ssm = boto3.client("ssm", "us-east-1")

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)

In [12]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [14]:
index_name = "genai-poc-knox-incremental-1024c-256o-v7"

### OpenSearch Client 생성

In [15]:
from utils.opensearch import opensearch_utils

In [16]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 4. Retriever based on Hybrid Search 정의
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- 본 샘플코드 에서는 **Hybrid Search based Retriever**를 **정의**합니다. 

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) Sematic serch를 통해 각 document별 relevant score 산출
- (2) Lexical search를 통해 각 document별 relevant score 산출
- (3-1) Rank-fusion 방식이 "simple weighted" 일 경우
    - 산출된 score에 대한 normalization 수행
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (3-2) Rank-fusion 방식이 "Reciprocal Rank Fusion (RRF)" 일 경우
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](../../../10_advanced_question_answering/img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

In [29]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [30]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    # necessary
    os_client=os_client,
    index_name=index_name,
    llm_emb=llm_emb,
    llm_text=llm_text,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.5, .5], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

### Retrieval example
- default search

In [31]:
from utils.rag import show_context_used

In [32]:
query = "knox"

In [33]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)


==========  Results  ==========

1. question: knox
2. # documents: 5
3. Documents: 

-----------------------------------------------
1. Chunk: 1002 Characters
-----------------------------------------------
Knox Manage release notes. Knox Manage 23.09 release notes Knox Manage 23.06 release notes Knox
Manage 23.03 release notes Knox Manage 22.11 release notes Knox Manage 22.08 release notes Knox
Manage 22.05 release notes Knox Manage 22.03 release notes Knox Manage 21.11 release notes Knox
Manage 21.09 release notes Knox Manage 21.07 release notes Knox Manage 21.06 release notes Knox
Manage 21.04 release notes Knox Manage 21.02 release notes Knox Manage 21.01 release notes Knox
Manage 20.11 release notes Knox Manage 20.08 release notes Knox Manage 20.06 release notes Knox
Manage 20.04 release notes Knox Manage 20.02 release notes Knox Manage 19.12 release notes Knox
Manage 19.11 release notes Knox Manage 19.09 release notes Knox Manage 19.05 release notes Knox
Manage 19.04 release not

- update parameters

In [37]:
opensearch_hybrid_retriever.update_search_params(
    k=10,
    minimum_should_match=0,
    filter=[],
    #filter=[
    #    {"term": {"metadata.seq_num": "64"}},
    #],
    async_mode=True
)

In [38]:
query = "knox"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)


==========  Results  ==========

1. question: knox
2. # documents: 10
3. Documents: 

-----------------------------------------------
1. Chunk: 1002 Characters
-----------------------------------------------
Knox Manage release notes. Knox Manage 23.09 release notes Knox Manage 23.06 release notes Knox
Manage 23.03 release notes Knox Manage 22.11 release notes Knox Manage 22.08 release notes Knox
Manage 22.05 release notes Knox Manage 22.03 release notes Knox Manage 21.11 release notes Knox
Manage 21.09 release notes Knox Manage 21.07 release notes Knox Manage 21.06 release notes Knox
Manage 21.04 release notes Knox Manage 21.02 release notes Knox Manage 21.01 release notes Knox
Manage 20.11 release notes Knox Manage 20.08 release notes Knox Manage 20.06 release notes Knox
Manage 20.04 release notes Knox Manage 20.02 release notes Knox Manage 19.12 release notes Knox
Manage 19.11 release notes Knox Manage 19.09 release notes Knox Manage 19.05 release notes Knox
Manage 19.04 release no

## 5. RAG using RetrievalQA powered by LangChain

In [40]:
from utils.rag import prompt_repo
from utils.rag import run_RetrievalQA
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [42]:
PROMPT = prompt_repo.get_qa(prompt_type="answer_with_ref") # ["answer_only", "answer_with_ref"]
pprint (PROMPT.template)

('\n'
 '            \n'
 '\n'
 'Human:\n'
 '            You are a mater answer bot designed to answer software '
 "developer's questions.\n"
 "            I'm going to give you a context. Read the context carefully, "
 "because I'm going to ask you a question about it.\n"
 '\n'
 '            Here is the context: <context>{context}</context>\n'
 '\n'
 '            First, find the paragraphs or sentences from the context that '
 'are most relevant to answering the question, and then print them in numbered '
 'order.\n'
 '            The format of paragraphs or sentences to the question should '
 "look like what's shown between the <references></references> tags.\n"
 '            Make sure to follow the formatting and spacing exactly.\n'
 '\n'
 '            <references>\n'
 '            [Examples of question + answer pairs using parts of the given '
 'context, with answers written exactly like how Claude’s output should be '
 'structured]\n'
 '            </references>\n'
 '\n'
 '        

### Update Search Params (Optional)

In [43]:
from langchain.chains import RetrievalQA

In [44]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=3,
    #filter=[
    #    {"term": {"metadata.project": "KS"}},
    #],
    filter=[],
    async_mode=True
)

### Request

In [45]:
qa = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [46]:
query = "What version of Knox Capture introduced the new AR demo mode feature?"
response = qa(query)

 <references>
1. Knox Capture v1.9 introduces the following new features: #Support for AR demo mode You can now test the Knox Capture AR feature without having to set up a real server or go through any profile creation or setup process.
</references>

<answer>
Knox Capture v1.9 introduced the new AR demo mode feature.
</answer>

In [47]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(response['source_documents'])

##################################
query:  What version of Knox Capture introduced the new AR demo mode feature?
##################################


### Answer ###
 <references>
1. Knox Capture v1.9 introduces the following new features: #Support for AR demo mode You can now
test the Knox Capture AR feature without having to set up a real server or go through any profile
creation or setup process.
</references>

<answer>
Knox Capture v1.9 introduced the new AR demo mode feature.
</answer>


### Contexts ###
-----------------------------------------------
1. Chunk: 990 Characters
-----------------------------------------------
Demo AR. As an IT admin, you can demo the Knox Capture AR features without having to set up a real
server or go through any profile configuration or setup process. AR demo mode only supports the
View-and-update template. To learn more about templates, see AR overview. Before you begin Before
you can demo the AR features, you'll first need to display the Knox Capt